In [1]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    MeshcatVisualizer,
    Parser,
    RigidTransform,
    RotationMatrix,
    StartMeshcat,
)

from manipulation.scenarios import SetColor

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [3]:
def grasp_poses_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.1)
    parser = Parser(plant, scene_graph)
    parser.SetAutoRenaming(True)

    '''
    grasp = parser.AddModelsFromUrl(
        #"package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
        "package://drake_models/atlas/robotiq.urdf"
        #"package://drake_models/atlas/robotiq_simple.urdf"
    )[0]
    pregrasp = parser.AddModelsFromUrl(
        "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    )[0]
    brick = parser.AddModelsFromUrl(
        "package://drake/examples/manipulation_station/models/061_foam_brick.sdf"
    )[0]
    '''
    
    '''grasp = parser.AddModelsFromUrl(
        #"package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
        "package://drake/examples/simple_gripper/robotiq_85_gripper.urdf"
        #"package://drake_models/atlas/robotiq_simple.urdf"
    )[0]
    pregrasp = parser.AddModelsFromUrl(
        #"package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
        "package://drake/examples/simple_gripper/robotiq_85_gripper.urdf"
        #"package://drake_models/atlas/robotiq_simple.urdf"
    )[0]'''
    
    grasp = parser.AddModels("/home/dan/Projects/Pydrake-robotiq140/models/robotiq_85_gripper_xuchenhan-tri.urdf")[0]
    pregrasp = parser.AddModels("/home/dan/Projects/Pydrake-robotiq140/models/robotiq_85_gripper_xuchenhan-tri.urdf")[0]
    brick = parser.AddModelsFromUrl(
        "package://drake/examples/manipulation_station/models/061_foam_brick.sdf"
    )[0]
    
    #brick = parser.AddModels("/home/dan/Projects/Pydrake-robotiq140/models/010_potted_meat_can.sdf")[0]
    
    
    plant.Finalize()

    B_O = plant.GetBodyByName("base_link", brick)
    B_Ggrasp = plant.GetBodyByName("robotiq_coupler_link", grasp)
    B_Gpregrasp = plant.GetBodyByName("robotiq_coupler_link", pregrasp)

    # Set the pregrasp to be green and slightly transparent.
    inspector = scene_graph.model_inspector()
    for body_index in plant.GetBodyIndices(pregrasp):
        SetColor(
            scene_graph,
            [0, 0.6, 0, 0.5],
            plant.get_source_id(),
            inspector.GetGeometries(plant.GetBodyFrameIdOrThrow(body_index)),
        )

    meshcat.Delete()
    meshcat.SetProperty("/Background", "visible", False)
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # TODO(russt): Set a random pose of the object.

    # Get the current object, O, pose
    X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)

    p_GgraspO = [0, 0.11, 0]
    R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2.0).multiply(
        RotationMatrix.MakeZRotation(np.pi / 2.0)
    )
    X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
    X_OGgrasp = X_GgraspO.inverse()
    X_WGgrasp = X_WO.multiply(X_OGgrasp)

    # pregrasp is negative y in the gripper frame (see the figure!).
    X_GgraspGpregrasp = RigidTransform([0, -0.08, 0])
    X_WGpregrasp = X_WGgrasp @ X_GgraspGpregrasp

    plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
    # Open the fingers, too.
    
    # The desired angle in radians to be stored in context.
    # 10 degress = 0.174533 approx. 0.175
    plant.GetJointByName("gripper_finger1_joint", grasp).set_angle(
        plant_context, 0.523599
    )
    
    '''plant.GetJointByName("right_inner_finger_joint", grasp).set_angle(
        plant_context, 0.349066
    )'''
    
    
    
    '''plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(
        plant_context, 0.054
    )'''

    plant.SetFreeBodyPose(plant_context, B_Gpregrasp, X_WGpregrasp)
    # Open the fingers, too.
    '''plant.GetJointByName("right_inner_finger_joint", pregrasp).set_translation(
        plant_context, -0.054
    )'''
    ''''plant.GetJointByName("right_finger_sliding_joint", pregrasp).set_translation(
        plant_context, 0.054
    )'''

    diagram.ForcedPublish(context)


grasp_poses_example()

### Example warning
# WARNING:drake:/home/dan/Projects/Pydrake-robotiq140/catkin_ws/src/robotiq/robotiq_2f_140_gripper_visualization/urdf/robotiq_arg2f_140_model.urdf:264: warning: Joint 'left_inner_knuckle_joint' specifies a mimic element that will be ignored. Mimic elements are currently only supported by MultibodyPlant with a discrete time step and using DiscreteContactSolver::kSap.
##https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html#a3296be9fa9bc35e6639460f54029c4c9

RuntimeError: /home/dan/Projects/Pydrake-robotiq140/models/robotiq_85_gripper_xuchenhan-tri.urdf:10: error: URI 'package://movo_description/meshes/manipulation/robotiq/visual/kinova_robotiq_coupler.obj' refers to unknown package 'movo_description'